In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_rumours.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [8]:
x_data = obtain_SWNUNetwork_input(
    method = "umap",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings.numpy(),
    k=5,
    time_feature='time_encoding',
    standardise_method=None)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [00:12<00:00, 462.87it/s]


[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 416])

In [25]:
df_rumours.iloc[50:54]

,id,label,datetime,text,timeline_id,timestamp
50,5.250724e+17,0,2014-10-22 23:53:11,@evening_grace @Telegraph Yes it is.,4,2014.808206
51,5.249251e+17,0,2014-10-22 14:07:39,BREAKING NEWS: Soldier shot at National War Me...,5,2014.807092
52,5.249253e+17,0,2014-10-22 14:08:48,@CBCNews 😱 why!!!,5,2014.807094
53,5.249256e+17,0,2014-10-22 14:09:39,@CBCNews,5,2014.807096


In [26]:
x_data[0][51,:,:3]

tensor([[2014.8073,    9.6880,    8.9186],
        [2014.8073,    9.9096,    8.7540],
        [2014.8073,    9.7161,    9.6193],
        [2014.8073,    9.6007,    9.5653],
        [2014.8073,    9.8437,    9.1731]])

In [24]:
x_data[0][3,:,:4]

tensor([[2015.0178,   10.0727,    9.1088,    4.1221],
        [2015.0178,    9.8144,    9.1319,    2.3864],
        [2015.0182,    9.8470,    9.1567,    2.7652],
        [2015.2263,   10.0102,    8.7004,    2.7885],
        [2015.2263,   10.0250,    8.7571,    2.8546]])

In [35]:
x_data[0][0,:,:1]

tensor([[2014.8076],
        [   0.0000],
        [   0.0000],
        [   0.0000],
        [   0.0000]])

SWNU Network

Obtaining path by looking at post history
We can obtain a path by looking at the history of each post. Here we look at the last 20 posts (and pad with vectors of zeros if there are less than 20 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [9]:
#dimensionality reduction
embedding_dim = sbert_embeddings.shape[1]
dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
dimensions = [30]#[50,30,15]
#time features
time_features = "time_encoding"
standardise_method = "standardise"
#SWNU block
log_sig = True
conv_output_channels = [10] #[20, 10, 5]
log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
bidirectional = False
#ffn
hidden_dim_sizes = [54]#[32,64]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
#overall training
num_epochs = 100
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [10]:
size=20
swnu_network_log_signature, best_swnu_network_log_signature, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dim_reduce_method,
    dimensions=dimensions, 
    sig_depths=[x[1] for x in log_signature_dimensions_and_sig_depths], 
    log_signature=log_sig,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes=[x[0] for x in log_signature_dimensions_and_sig_depths],
    ffn_hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    time_feature=time_features, 
    standardise_method=standardise_method,
    path_indices=None,
    data_split_seed=123,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]



##################################################
dimension: 30 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


























































































































































































































































































































100%|██████████| 5568/5568 [00:18<00:00, 308.11it/s]


[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


Accuracy on dataset of size 891: 65.0954008102417 %.
Average loss: 0.22366737255028316
- f1: [0.70239234 0.578019  ]
- f1 (macro): 0.6402056702135257


  0%|          | 0/1 [02:34<?, ?it/s]


KeyboardInterrupt: 